# Learning to play Vegas-style Solitaire with AI

---

Author: Andrew Vadnais

Description: Readme in repo

First, some import statements:

In [7]:
import numpy as np

## Card

In order to save ourselves from messy and repeated code below, we'll create a class for a playing card, which hold the number of the card and an int, and the suit of the card as a string seperately.  This is done as opposed to storing each card as a string, and constantly manipulating the string and converting it from string to int to do comparisions and prints.

In [67]:
class Card:
    def __init__(self,number,suit):
        self.num = number
        self.suit = suit
        
    def to_str(self):
        if(self.num)==1:
            return " A"+self.suit
        elif(self.num)==10:
            return str(self.num)+self.suit
        elif(self.num)==11:
            return " J"+self.suit
        elif(self.num)==12:
            return " Q"+self.suit
        elif(self.num)==13:
            return " K"+self.suit
        else:
            return " "+str(self.num)+self.suit
    
    def __repr__(self):
        return self.to_str()

## State

To uniquely describe a single point in the game of Klondike, we need to know a few things.

$\cdot$ the top card in each of the foundation stacks

$\cdot$ the full sequence in each of the playing stacks

$\cdot$ the drawing stack

$\cdot$ the current location in the drawing stack

Therefore, our state class should include all of these things.  In addition, we should include some kind of rating of how good this round is -- by keeping it general, we can investigate the usefulness of certain tactics later on.  We'll also include an extensive representation method in this class, to make any print-outs of the board pretty.

In [68]:
class State:
    def __init__(self,foundation,playing,drawing,drawing_location=0):
        self.found = foundation
        self.play = playing
        self.draw = drawing
        self.loc = drawing_location
        self.known = [x for x in range(7)]
        self.run = [x for x in range(7)]
        self.obj = None
        
    def __repr__(self):
        printOut = self.found['h'].to_str()+" "+self.found['c'].to_str()+" "+self.found['d'].to_str()+" "+self.found['s'].to_str()+"\n"
        
        max_len = np.max([len(stack) for stack in self.play])
        for row in range(max_len):
            row_curr = []
            for stack in range(7):
                if(len(self.play[stack])>row):
                    if(row<self.known[stack]):
                        card = "---"
                    else:
                        card = self.play[stack][row].to_str()
                    row_curr.append(card)
                else:
                    row_curr.append("   ")
                    
            printOut += row_curr[0]+" " +row_curr[1]+" " +row_curr[2]+" "
            printOut += row_curr[3]+" " +row_curr[4]+" " +row_curr[5]+" "
            printOut += row_curr[6]+"\n"
            
        printOut += "          "+self.draw[self.loc].to_str()+" "+self.draw[self.loc+1].to_str()+" "+self.draw[self.loc+2].to_str()+"\n"
        
        printOut += "\nObjective Value: "+str(self.obj)

        return printOut

For the sake of simplicity, we'll define a function here to return a random initial state.

In [69]:
def initial_state(objective_func):
    cards = np.array([Card(num, suit) for num in range(1,14) for suit in ['h','c','d','s']])
    cards_shuff = [i for i in range(len(cards))]
    np.random.shuffle(cards_shuff)
    deck_loc = 0
    
    foundation = {suit:Card(0,suit) for suit in ['h','c','d','s']}
    play = [[] for ii in range(7)]
    
    #dealing under vegas regulations - each new card from the top of the deck
    for row in range(7):
        for stack in range(row,7):
            play[stack].append(cards[cards_shuff[deck_loc]])
            deck_loc += 1

    drawing = cards[cards_shuff[deck_loc:]]
    
    state_init = State(foundation=foundation, playing=play, drawing=drawing)
    state_init.obj = objective_func(state_init)
    
    return state_init

We're gonna create a simple objective function to test the above initial state creation.  We'll just use the number of cards in our foundation stacks (that is, we expect 0 to be returned)

In [72]:
def found_obj(curr):
    found_val = [stack.num if stack.num!="__" else 0 for stack in curr.found.values()]
    return np.sum(found_val)

state_test = initial_state(found_obj)
print(state_test,"\n")

state_test.found = {suit:Card(num,suit) for num,suit in zip([3,3,4,3],['h','c','d','s'])}
state_test.obj = found_obj(state_test)
print(state_test)

 0h  0c  0d  0s
 6h --- --- --- --- --- ---
     8h --- --- --- --- ---
         5h --- --- --- ---
             7d --- --- ---
                 Kc --- ---
                     3h ---
                         9s
          10c  5c  3s

Objective Value: 0 

 3h  3c  4d  3s
 6h --- --- --- --- --- ---
     8h --- --- --- --- ---
         5h --- --- --- ---
             7d --- --- ---
                 Kc --- ---
                     3h ---
                         9s
          10c  5c  3s

Objective Value: 13


## Game

Now that we can describe every possible point in a game, we can actually play one!  In order to do this, we'll need to know:

$\cdot$ The current state that the game is in

$\cdot$ Whether we've made any moves in the current pass through the drawing stack

$\cdot$ Whether we've won

In addition to knowing the above, we'll also want some way to do the following:

$\cdot$ Find all possible moves from the current state

$\cdot$ Find the best possible move based on our heuristic/loss/objective function

$\cdot$ Change from one state to another, described by some move

$\cdot$ Test whether or not the game is over

Moves will be described as a list composed of [location of card, destination of card].  We'll include all of the above in our Game class.

In [79]:
class Klondike_Game:
    def __init__(self, current_state):
        self.curr = current_state
        self.isWon = False
        self.madeMove = False
        
    def moves(self):
        '''
        return: a list of all possible moves from the current state 
        '''
        print(self.curr)
        color = {'h':'red','d':'red','c':'black','s':'black'}
        moves = []
        stack_num = [x for x in range(7)]
        for stack in stack_num:
            #search for moves from the top of a run to another stack
            top = self.curr.play[stack][self.curr.run[stack]]
            bottom = self.curr.play[stack][-1]
            stack_num.remove(stack)
            for space in stack_num:
                if(self.curr.play[space][-1].num==top.num+1):
                    if(color[self.curr.play[space][-1].suit]!=color[top.suit]):
                        moves.append(['stacks',stack,space])
                        
            #search for moves from the bottom of a stack to the foundation stacks
            if(bottom.num==self.curr.found[bottom.suit].num+1):
                moves.append(['found',stack])
                
            #search for a move from the top of the draw to a stack
            if(bottom.num==self.curr.draw[self.curr.loc].num+1):
                if(color[bottom.suit]!=color[self.curr.draw[self.curr.loc].suit]):
                    moves.append(['draw',stack])
            stack_num.insert(stack,stack)
        return moves

In [84]:
Klondike_Game(initial_state(found_obj)).moves()

 0h  0c  0d  0s
 6s --- --- --- --- --- ---
    10d --- --- --- --- ---
         Qc --- --- --- ---
             3c --- --- ---
                 7d --- ---
                     4s ---
                         Ks
           2d  8h  5c

Objective Value: 0


[['stacks', 0, 4], ['draw', 3]]